# Parquet File Analysis for DuckDB-WASM Optimization

This notebook analyzes parquet files to check:
- Bloom filters presence
- Statistics (min/max) quality
- Row group sizes
- Sorting order
- Compression

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa
import os
from pathlib import Path
import json

## Configuration

In [ ]:
# Path to parquet files
PARQUET_DIR = Path("../input_data/vulnerable_origins")

# Get first few files for analysis
parquet_files = sorted(PARQUET_DIR.glob("*.parquet"))[:5]
print(f"Found {len(list(PARQUET_DIR.glob('*.parquet')))} total parquet files")
print(f"Analyzing first {len(parquet_files)} files:")
for f in parquet_files:
    size_mb = os.path.getsize(f) / 1024 / 1024
    print(f"  - {f.name}: {size_mb:.1f} MB")

## 1. Basic File Info

In [ ]:
def analyze_file_basic(filepath):
    """Get basic file information"""
    pf = pq.ParquetFile(filepath)
    meta = pf.metadata
    
    return {
        'file': filepath.name,
        'size_mb': os.path.getsize(filepath) / 1024 / 1024,
        'num_rows': meta.num_rows,
        'num_row_groups': meta.num_row_groups,
        'num_columns': meta.num_columns,
        'created_by': meta.created_by,
        'format_version': meta.format_version,
    }

for f in parquet_files:
    info = analyze_file_basic(f)
    print(f"\n{'='*60}")
    print(f"File: {info['file']}")
    print(f"{'='*60}")
    print(f"Size: {info['size_mb']:.2f} MB")
    print(f"Rows: {info['num_rows']:,}")
    print(f"Row Groups: {info['num_row_groups']}")
    print(f"Columns: {info['num_columns']}")
    print(f"Created by: {info['created_by']}")
    print(f"Format version: {info['format_version']}")

## 2. Schema Analysis

In [ ]:
# Analyze schema of first file
pf = pq.ParquetFile(parquet_files[0])
print("Schema:")
print(pf.schema_arrow)
print("\nParquet Schema:")
print(pf.schema)

## 3. Bloom Filter Check

Bloom filters allow DuckDB to quickly skip row groups that definitely don't contain a value.
Without bloom filters, DuckDB must download and scan row groups to check.

In [ ]:
def check_bloom_filters(filepath):
    """Check if bloom filters exist in the parquet file"""
    pf = pq.ParquetFile(filepath)
    meta = pf.metadata
    
    results = []
    
    # Check first few row groups
    for rg_idx in range(min(3, meta.num_row_groups)):
        rg = meta.row_group(rg_idx)
        
        for col_idx in range(meta.num_columns):
            col = rg.column(col_idx)
            
            # Try to detect bloom filter
            # PyArrow doesn't expose bloom_filter_offset directly in newer versions
            # We check via the column metadata
            col_dict = col.to_dict()
            
            has_bloom = 'bloom_filter_offset' in col_dict and col_dict.get('bloom_filter_offset') is not None
            
            if rg_idx == 0:  # Only report once per column
                results.append({
                    'column': col.path_in_schema,
                    'has_bloom_filter': has_bloom,
                    'bloom_filter_offset': col_dict.get('bloom_filter_offset'),
                    'bloom_filter_length': col_dict.get('bloom_filter_length'),
                })
    
    return results

print("Bloom Filter Analysis")
print("="*60)

for f in parquet_files[:1]:  # Check first file
    print(f"\nFile: {f.name}")
    results = check_bloom_filters(f)
    
    has_any_bloom = any(r['has_bloom_filter'] for r in results)
    
    if has_any_bloom:
        print("✓ Bloom filters FOUND")
    else:
        print("✗ NO bloom filters - queries will be slow!")
    
    print("\nPer-column details:")
    for r in results:
        status = "✓" if r['has_bloom_filter'] else "✗"
        print(f"  {status} {r['column']}: bloom_offset={r['bloom_filter_offset']}")

## 4. Statistics Analysis

Min/max statistics allow DuckDB to skip row groups where the searched value is outside the range.
For this to work well, data should be SORTED by the query column.

In [ ]:
def analyze_statistics(filepath, column_name='origin', num_row_groups=10):
    """Analyze statistics for a specific column across row groups"""
    pf = pq.ParquetFile(filepath)
    meta = pf.metadata
    
    # Find column index
    col_idx = None
    for i in range(meta.num_columns):
        if meta.row_group(0).column(i).path_in_schema == column_name:
            col_idx = i
            break
    
    if col_idx is None:
        print(f"Column '{column_name}' not found")
        return
    
    print(f"Statistics for column '{column_name}':")
    print("-" * 80)
    
    stats_list = []
    for rg_idx in range(min(num_row_groups, meta.num_row_groups)):
        rg = meta.row_group(rg_idx)
        col = rg.column(col_idx)
        stats = col.statistics
        
        if stats and stats.has_min_max:
            min_val = str(stats.min)[:50]
            max_val = str(stats.max)[:50]
            stats_list.append({'min': stats.min, 'max': stats.max})
            print(f"RG {rg_idx:3d}: min='{min_val}' | max='{max_val}'")
        else:
            print(f"RG {rg_idx:3d}: NO STATISTICS")
    
    # Check if data appears sorted
    if len(stats_list) > 1:
        is_sorted = all(
            stats_list[i]['max'] <= stats_list[i+1]['min'] 
            for i in range(len(stats_list)-1)
        )
        
        # Check overlap
        overlaps = sum(
            1 for i in range(len(stats_list)-1)
            if stats_list[i]['max'] > stats_list[i+1]['min']
        )
        
        print("\n" + "="*80)
        if is_sorted:
            print("✓ Data appears SORTED - statistics will be effective!")
        else:
            print(f"✗ Data NOT sorted - {overlaps} overlapping row groups")
            print("  → Most row groups will need to be downloaded")

# Analyze first file
analyze_statistics(parquet_files[0], 'origin', num_row_groups=10)

## 5. Row Group Size Analysis

Smaller row groups = more granular filtering, but more metadata overhead.
Recommended: 100K-500K rows per row group for browser queries.

In [ ]:
def analyze_row_groups(filepath):
    """Analyze row group sizes"""
    pf = pq.ParquetFile(filepath)
    meta = pf.metadata
    
    sizes = []
    rows = []
    
    for rg_idx in range(meta.num_row_groups):
        rg = meta.row_group(rg_idx)
        sizes.append(rg.total_byte_size)
        rows.append(rg.num_rows)
    
    avg_size_mb = sum(sizes) / len(sizes) / 1024 / 1024
    avg_rows = sum(rows) / len(rows)
    min_size_mb = min(sizes) / 1024 / 1024
    max_size_mb = max(sizes) / 1024 / 1024
    
    print(f"Row Group Analysis for {filepath.name}")
    print("="*60)
    print(f"Total row groups: {len(sizes)}")
    print(f"Avg rows per group: {avg_rows:,.0f}")
    print(f"Avg size per group: {avg_size_mb:.2f} MB")
    print(f"Min size: {min_size_mb:.2f} MB")
    print(f"Max size: {max_size_mb:.2f} MB")
    
    # Recommendation
    print("\nRecommendation:")
    if avg_rows > 500_000:
        print(f"  ⚠ Row groups are large ({avg_rows:,.0f} rows)")
        print("  → Consider smaller row groups (100K-500K) for finer filtering")
    elif avg_rows < 50_000:
        print(f"  ⚠ Row groups are small ({avg_rows:,.0f} rows)")
        print("  → May have too much metadata overhead")
    else:
        print(f"  ✓ Row group size looks good ({avg_rows:,.0f} rows)")

for f in parquet_files[:2]:
    analyze_row_groups(f)
    print()

## 6. Compression Analysis

In [ ]:
def analyze_compression(filepath):
    """Analyze compression settings"""
    pf = pq.ParquetFile(filepath)
    meta = pf.metadata
    
    rg = meta.row_group(0)
    
    print(f"Compression Analysis for {filepath.name}")
    print("="*60)
    
    for col_idx in range(meta.num_columns):
        col = rg.column(col_idx)
        compressed = col.total_compressed_size
        uncompressed = col.total_uncompressed_size
        ratio = uncompressed / compressed if compressed > 0 else 0
        
        print(f"{col.path_in_schema}:")
        print(f"  Compression: {col.compression}")
        print(f"  Encodings: {col.encodings}")
        print(f"  Ratio: {ratio:.1f}x ({compressed/1024:.1f} KB → {uncompressed/1024:.1f} KB)")
        print()

analyze_compression(parquet_files[0])

## 7. Summary & Recommendations

In [ ]:
def full_analysis(filepath):
    """Complete analysis with recommendations"""
    pf = pq.ParquetFile(filepath)
    meta = pf.metadata
    
    issues = []
    
    # Check file size
    size_mb = os.path.getsize(filepath) / 1024 / 1024
    if size_mb > 500:
        issues.append(f"File too large ({size_mb:.0f} MB) - browser may run out of memory")
    
    # Check bloom filters
    rg = meta.row_group(0)
    col = rg.column(0)  # Check first column (origin)
    col_dict = col.to_dict()
    if not col_dict.get('bloom_filter_offset'):
        issues.append("No bloom filters - queries will download unnecessary data")
    
    # Check sorting
    stats_list = []
    for rg_idx in range(min(10, meta.num_row_groups)):
        rg = meta.row_group(rg_idx)
        col = rg.column(0)
        if col.statistics and col.statistics.has_min_max:
            stats_list.append({'min': col.statistics.min, 'max': col.statistics.max})
    
    if len(stats_list) > 1:
        overlaps = sum(
            1 for i in range(len(stats_list)-1)
            if stats_list[i]['max'] > stats_list[i+1]['min']
        )
        if overlaps > 0:
            issues.append(f"Data not sorted by 'origin' - {overlaps} overlapping row groups")
    
    # Check row group size
    avg_rows = meta.num_rows / meta.num_row_groups
    if avg_rows > 500_000:
        issues.append(f"Row groups too large ({avg_rows:,.0f} rows) - use 100K-500K")
    
    return issues

print("="*80)
print("PARQUET OPTIMIZATION SUMMARY")
print("="*80)

all_issues = []
for f in parquet_files:
    issues = full_analysis(f)
    all_issues.extend(issues)
    
    print(f"\n{f.name}:")
    if issues:
        for issue in issues:
            print(f"  ✗ {issue}")
    else:
        print("  ✓ All checks passed!")

if all_issues:
    print("\n" + "="*80)
    print("RECOMMENDED ACTIONS:")
    print("="*80)
    print("""
Run the optimization script to fix these issues:

    python scripts/optimizeParquet.py input_data/vulnerable_origins output_data/vulnerable_origins

This will:
1. Sort data by 'origin' for better statistics pruning
2. Add bloom filters for fast negative lookups  
3. Use smaller row groups (100K rows) for finer filtering
4. Split large files to avoid browser memory issues
""")

## 8. Test Query Simulation

Simulate how many row groups would need to be downloaded for a specific query.

In [ ]:
def simulate_query(filepath, search_value='https://github.com/django/django'):
    """Simulate how many row groups would be downloaded for a query"""
    pf = pq.ParquetFile(filepath)
    meta = pf.metadata
    
    # Find origin column
    col_idx = 0  # Assuming origin is first column
    
    would_download = 0
    would_skip_stats = 0
    would_skip_bloom = 0
    
    total_bytes_download = 0
    total_bytes_skip = 0
    
    for rg_idx in range(meta.num_row_groups):
        rg = meta.row_group(rg_idx)
        col = rg.column(col_idx)
        stats = col.statistics
        
        # Check if statistics would skip this row group
        if stats and stats.has_min_max:
            if search_value < stats.min or search_value > stats.max:
                would_skip_stats += 1
                total_bytes_skip += rg.total_byte_size
                continue
        
        # If we get here, we'd need to download (bloom filter would help but we can't check it here)
        would_download += 1
        total_bytes_download += rg.total_byte_size
    
    print(f"Query Simulation: WHERE origin = '{search_value}'")
    print(f"File: {filepath.name}")
    print("="*60)
    print(f"Total row groups: {meta.num_row_groups}")
    print(f"Would skip (via stats): {would_skip_stats} ({would_skip_stats/meta.num_row_groups*100:.1f}%)")
    print(f"Would download: {would_download} ({would_download/meta.num_row_groups*100:.1f}%)")
    print(f"\nData transfer:")
    print(f"  Would download: {total_bytes_download/1024/1024:.1f} MB")
    print(f"  Would skip: {total_bytes_skip/1024/1024:.1f} MB")
    
    if would_download > meta.num_row_groups * 0.1:
        print(f"\n⚠ WARNING: Would download {would_download/meta.num_row_groups*100:.0f}% of row groups!")
        print("  → Data needs to be sorted by 'origin' for efficient queries")

# Test with a sample query
simulate_query(parquet_files[0], 'https://github.com/django/django')